In [1]:
import json, logging, os, re, requests, sys, time
import pandas as pd

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from datetime import date, datetime
from pykrx import stock
from webob.compat import urlparse
import lxml
URL_NAVER_FINANCE_NEWS_QUERY = "https://finance.naver.com/news/news_search.nhn?rcdate=&q=%s&x=0&y=0&sm=all.basic&pd=4&stDateStart=%s&stDateEnd=%s&page=%s"
URL_NAVER_FINANCE = "http://finance.naver.com"

In [2]:
comp_name = stock.get_market_ticker_name('019170')
print("comp name = ", comp_name)

comp name =  신풍제약


In [3]:
q = ""
for c in comp_name.encode("euc-kr"):
    q += '%%%s' % format(c, "x").capitalize()
    print(q)

%Bd
%Bd%C5
%Bd%C5%C7
%Bd%C5%C7%B3
%Bd%C5%C7%B3%C1
%Bd%C5%C7%B3%C1%A6
%Bd%C5%C7%B3%C1%A6%Be
%Bd%C5%C7%B3%C1%A6%Be%E0


In [4]:
page = 1
n_news = 0

In [5]:
r_url = URL_NAVER_FINANCE_NEWS_QUERY % (q, '2022-03-29', '2022-03-30', page,)

In [6]:
r = requests.get(r_url)

In [7]:
soup = bs(r.text,'lxml')

In [8]:
news = soup.find('div', {'class':'newsSchResult'}).find('dl',{'class':'newsList'})

In [9]:
news_title = news.find_all('dd',{'class':'articleSubject'})
news_title

[<dd class="articleSubject">
 <a href="/news/news_read.naver?article_id=0005066323&amp;office_id=277&amp;mode=search&amp;query=신풍제약&amp;page=1">코스피, 기관 매수세에 상승 마감…2750선 안착에는 실패</a>
 </dd>,
 <dd class="articleSubject">
 <a href="/news/news_read.naver?article_id=0002589520&amp;office_id=119&amp;mode=search&amp;query=신풍제약&amp;page=1">라게브리오 도입…일동제약, 신풍제약 먹는 코로나약 경쟁력은</a>
 </dd>,
 <dd class="articleSubject">
 <a href="/news/news_read.naver?article_id=0004725985&amp;office_id=008&amp;mode=search&amp;query=신풍제약&amp;page=1">종근당건강 건기식서 대박나자...너도나도 캐시카우 노린다</a>
 </dd>,
 <dd class="articleSubject">
 <a href="/news/news_read.naver?article_id=0004035483&amp;office_id=011&amp;mode=search&amp;query=신풍제약&amp;page=1">신풍제약에 고수익 투자자 매수세</a>
 </dd>]

In [10]:
news_summary = news.find_all('dd',{'class':'articleSummary'})

In [11]:
wdate = news.find_all('span', {'class':'wdate'})

In [12]:
n_news += len(news_title)

In [13]:
n_news

4

In [14]:
for title, summary, _date in zip(news_title, news_summary, wdate):
    date = _date.find(text=True).split("\n")[2].strip()
    news_title = title.find(text=True)
    print(news_title)
    break

In [16]:
title.a.text

'코스피, 기관 매수세에 상승 마감…2750선 안착에는 실패'

In [17]:
url = URL_NAVER_FINANCE + title.a.get("href")

In [18]:
summary.find(text=True).strip('\n\t\r')

'의약품업 중에선 일동제약(9.4%)과 신풍제약(4.8%)이 강세를 보였는데 코로나 치료제 개발 기대감이 반영됐다. 전일 미국 증시에서 자동차 주...'

In [20]:
urlparse.parse_qs(urlparse.urlparse(url).query)['article_id'][0]

'0005066323'

In [21]:
r = requests.get(url)
soup = bs(r.text, "lxml")

In [23]:
content = soup.find("div", class_="articleCont", id="content")
re.findall(r"\d{5}",content.text)

[]

In [25]:
content

<div class="articleCont" id="content">
<span class="end_photo_org"><img alt="" src="https://imgnews.pstatic.net/image/277/2022/03/30/0005066323_001_20220330155801505.jpg?type=w540"/><em class="img_desc">[이미지출처=연합뉴스]</em></span><br/><br/>코스피가 장단기 금리 역전에 따른 경기 침체 우려 등에도 상승 마감했다. 다만 장중 외국인이 순매도로 돌아서면서 2750선 안착에는 실패했다.<br/><br/>30일 코스피는 전 거래일 대비 0.21%(5.67포인트) 상승한 2746.74로 장을 끝냈다. 이날 코스피는 장 초반에는 0.6%가 넘는 오름세를 보였다. 전일 미국 증시가 장단기 금리 역전에 따른 경기침체 우려에도 러시아와 우크라이나의 휴전 기대감을 높게 반영하면서 상승 마감한 것이 긍정적으로 작용했다. 그러나 외국인이 순매도로 돌아서면서 장 막판엔 상승폭이 줄어들었다. <br/><br/>투자자동향을 보면 기관이 1604억원을 샀고 개인과 외국인이 각각 1376억원, 111억원어치 주식을 팔았다.<br/><br/>이경민 대신증권 연구원은 “3월 중순부터 이어진 글로벌 증시 상승세는 결국 기대감에 의존한 것”이라며 “국내 증시는 중국발 변수에 이어 일본 변수까지 신경을 써야하는 상황에서 국내 기업 실적 관련 불확실성도 유입돼 다른 아시아 증시 대비 부진한 모습을 보였다”고 설명했다.<br/><br/>이날 증시에선 의료정밀(1.87%), 의약품(1.83%), 운수장비(1.26%) 업종이 강세를 보였다. 의약품업 중에선 일동제약(9.4%)과 신풍제약(4.8%)이 강세를 보였는데 코로나 치료제 개발 기대감이 반영됐다. 전일 미국 증시에서 자동차 주가 강세를 보인 영향을 받아 자동차 관련주도 상승했다. 종목별로는 세종공업(7.4%) 기아(2.6%), 현대차(2.5%), 현대위아(2.4%) 등이

In [26]:
cntnt = content.text.strip('\n\t\r').split('@')[0]

In [27]:
effix = cntnt.split('.')[-1]
effix

'제일 먼저 임철수 떠올라"낮엔 굴삭기 기사, 밤에는 \'포주\'…40대 남성의 두 얼굴\'장애아 엄마\' 나경원, 이준석과 거리두기…" 조롱 미성숙해""6개월마다 새로운 변이 나온다"…유럽서 또 새 변이 발견'

In [28]:
cntnt.split('.')

['[이미지출처=연합뉴스]코스피가 장단기 금리 역전에 따른 경기 침체 우려 등에도 상승 마감했다',
 ' 다만 장중 외국인이 순매도로 돌아서면서 2750선 안착에는 실패했다',
 '30일 코스피는 전 거래일 대비 0',
 '21%(5',
 '67포인트) 상승한 2746',
 '74로 장을 끝냈다',
 ' 이날 코스피는 장 초반에는 0',
 '6%가 넘는 오름세를 보였다',
 ' 전일 미국 증시가 장단기 금리 역전에 따른 경기침체 우려에도 러시아와 우크라이나의 휴전 기대감을 높게 반영하면서 상승 마감한 것이 긍정적으로 작용했다',
 ' 그러나 외국인이 순매도로 돌아서면서 장 막판엔 상승폭이 줄어들었다',
 ' 투자자동향을 보면 기관이 1604억원을 샀고 개인과 외국인이 각각 1376억원, 111억원어치 주식을 팔았다',
 '이경민 대신증권 연구원은 “3월 중순부터 이어진 글로벌 증시 상승세는 결국 기대감에 의존한 것”이라며 “국내 증시는 중국발 변수에 이어 일본 변수까지 신경을 써야하는 상황에서 국내 기업 실적 관련 불확실성도 유입돼 다른 아시아 증시 대비 부진한 모습을 보였다”고 설명했다',
 '이날 증시에선 의료정밀(1',
 '87%), 의약품(1',
 '83%), 운수장비(1',
 '26%) 업종이 강세를 보였다',
 ' 의약품업 중에선 일동제약(9',
 '4%)과 신풍제약(4',
 '8%)이 강세를 보였는데 코로나 치료제 개발 기대감이 반영됐다',
 ' 전일 미국 증시에서 자동차 주가 강세를 보인 영향을 받아 자동차 관련주도 상승했다',
 ' 종목별로는 세종공업(7',
 '4%) 기아(2',
 '6%), 현대차(2',
 '5%), 현대위아(2',
 '4%) 등이 오름세를 보였다',
 '시가총액 상위종목을 보면 삼성전자(-0',
 '43%), LG에너지솔루션(-0',
 '11%)이 하락했고, SK하이닉스(0',
 '83%), NAVER(1',
 '50%), 삼성바이오로직스(1',
 '93%), 카카오(0',
 '95%), 삼성SDI(3',
 '51%)은 상

In [29]:
cntnt.replace(effix, '')

'[이미지출처=연합뉴스]코스피가 장단기 금리 역전에 따른 경기 침체 우려 등에도 상승 마감했다. 다만 장중 외국인이 순매도로 돌아서면서 2750선 안착에는 실패했다.30일 코스피는 전 거래일 대비 0.21%(5.67포인트) 상승한 2746.74로 장을 끝냈다. 이날 코스피는 장 초반에는 0.6%가 넘는 오름세를 보였다. 전일 미국 증시가 장단기 금리 역전에 따른 경기침체 우려에도 러시아와 우크라이나의 휴전 기대감을 높게 반영하면서 상승 마감한 것이 긍정적으로 작용했다. 그러나 외국인이 순매도로 돌아서면서 장 막판엔 상승폭이 줄어들었다. 투자자동향을 보면 기관이 1604억원을 샀고 개인과 외국인이 각각 1376억원, 111억원어치 주식을 팔았다.이경민 대신증권 연구원은 “3월 중순부터 이어진 글로벌 증시 상승세는 결국 기대감에 의존한 것”이라며 “국내 증시는 중국발 변수에 이어 일본 변수까지 신경을 써야하는 상황에서 국내 기업 실적 관련 불확실성도 유입돼 다른 아시아 증시 대비 부진한 모습을 보였다”고 설명했다.이날 증시에선 의료정밀(1.87%), 의약품(1.83%), 운수장비(1.26%) 업종이 강세를 보였다. 의약품업 중에선 일동제약(9.4%)과 신풍제약(4.8%)이 강세를 보였는데 코로나 치료제 개발 기대감이 반영됐다. 전일 미국 증시에서 자동차 주가 강세를 보인 영향을 받아 자동차 관련주도 상승했다. 종목별로는 세종공업(7.4%) 기아(2.6%), 현대차(2.5%), 현대위아(2.4%) 등이 오름세를 보였다.시가총액 상위종목을 보면 삼성전자(-0.43%), LG에너지솔루션(-0.11%)이 하락했고, SK하이닉스(0.83%), NAVER(1.50%), 삼성바이오로직스(1.93%), 카카오(0.95%), 삼성SDI(3.51%)은 상승했다.같은 시각 코스닥지수는 전 거래일 대비 0.13%(1.24포인트) 상승한 937.07로 장을 끝냈다. 투자자 동향을 보면 개인과 외국인이 각각 270억원, 233억원어치 주식을 샀고 기관은 홀로 296억원어치 주식을 팔아치웠

In [30]:
logger = logging.getLogger(__name__)

In [31]:
formatter = logging.Formatter(
            "%(asctime)s %(levelname)s [%(name)s] [%(filename)s:%(lineno)d] - %(message)s"
        )

In [32]:
formatter

In [33]:
streamHandler = logging.StreamHandler()

In [34]:
streamHandler.setFormatter(formatter)

In [35]:
logger.addHandler(streamHandler)

In [36]:
logger.setLevel(level=logging.INFO)

In [193]:
dt = '2022-03-30'
today = date.today().strftime("%Y-%m-%d")
print(f'today= {today}')
if dt is None:
    dt = today
else:
    dt = dt

AttributeError: 'str' object has no attribute 'today'

In [43]:
datetime.today().strftime("%Y-%m-%d")

'2022-03-30'

In [42]:
now()

NameError: name 'now' is not defined

In [44]:
_comp = {}

In [56]:
_comp["ticker"] = ['019170','005930']

In [48]:
def get_valid_ticker_name(ticker):
    try:
        return stock.get_market_ticker_name(ticker)
    except:
        pass

In [59]:
_comp["name"] = get_valid_ticker_name(['005930','019170'])

In [63]:
_comp["name"] = [get_valid_ticker_name(x) for x in ['005930','019170']]

In [64]:
_comp

{'ticker': ['019170', '005930'], 'name': ['삼성전자', '신풍제약']}

In [69]:
for i, t in enumerate(_comp["ticker"]):
    logger.info("i = {}, comp = {}".format(i, _comp["ticker"][i]))
    try:
        msg = "company = {}".format(_comp["name"][i])
        logger.info(msg)
        time.sleep(1)
        self._crawl_by_query(ticker=t, dt=dt)
    except Exception as e:
        logger.error(e)
        pass

2022-03-30 17:05:11,288 INFO [__main__] [482169060.py:2] - i = 0, comp = 019170
INFO:__main__:i = 0, comp = 019170
2022-03-30 17:05:11,290 INFO [__main__] [482169060.py:5] - company = 삼성전자
INFO:__main__:company = 삼성전자
2022-03-30 17:05:12,305 ERROR [__main__] [482169060.py:9] - name 'self' is not defined
ERROR:__main__:name 'self' is not defined
2022-03-30 17:05:12,306 INFO [__main__] [482169060.py:2] - i = 1, comp = 005930
INFO:__main__:i = 1, comp = 005930
2022-03-30 17:05:12,307 INFO [__main__] [482169060.py:5] - company = 신풍제약
INFO:__main__:company = 신풍제약
2022-03-30 17:05:13,318 ERROR [__main__] [482169060.py:9] - name 'self' is not defined
ERROR:__main__:name 'self' is not defined


In [67]:
msg = "company = {}".format(_comp["name"][i])

In [68]:
msg

'company = 신풍제약'